# Real or Not? NLP with Disaster Tweets (Kaggle Competition)

In [15]:
from IPython.display import YouTubeVideo
YouTubeVideo("", width=600)

[Link to the GitHub repository](https://github.com/XaviJunior/SBB)

[Link to the YouTube video]()

## Contributions

* **Xavier AEBY**: 
* **Tarik BACHA**: 
* **Tanguy BERGUERAND**: 
* **Frederic SPYCHER**: 

## Introduction

For our second group project, we were tasked with joining the [Real or Not? NLP with Disaster Tweets](https://www.kaggle.com/c/nlp-getting-started) Kaggle competition, which consists of devising a machine learning model that can predict, using natural language processing, whether tweets announcing a disaster are genuine or not.

The incentive for such a model is to assist disaster relief organizations and news agencies identify actual   as they monitor Twitter (which is often used as a communication tool during such events).

## Setting things up

In [0]:
# !pip install spacy
# !pip install xgboost

In [0]:
import re
import string

import numpy as np
import pandas as pd
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

RSEED = 42

## The data

The data provided by Kaggle contains more than **10,000 tweets**, for 70% of which we are given their `target` class (1 = true, 0 = false). Each observation from the training data is composed of an `id` and the `text` of the tweet. In some cases, a `keyword` as well as the tweet's `location` are also given.

In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv", encoding="utf-8")
df.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [19]:
print("Classified observations:", df.shape[0])

Classified observations: 7613


In [20]:
df_test = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv", encoding="utf-8")
df_test.head(3)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."


In [21]:
print("Unclassified observations:", df_test.shape[0])

Unclassified observations: 3263


## Cleaning

In [0]:
def clean(tweet):
    tweet = re.sub(r" \w{1,3}\.{3,3} http\S{0,}", " ", tweet)
    tweet = re.sub(r"http\S{0,}", " ", tweet)
    tweet = re.sub(r".Û.", "", tweet)
    tweet = re.sub(r"\s+", " ", tweet)
    tweet = re.sub(r'\t', " ", tweet)
    tweet = re.sub(r'\n', " ", tweet)
    tweet = re.sub(r"\.{3,3}", "", tweet)

    return tweet
    
df["text"] = df["text"].apply(clean)
df_test["text"] = df_test["text"].apply(clean)

# df["text"].to_csv("tweets.csv", index=False)

## Tokenization

In [0]:
nlp = English()
punctuation = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def tokenizer(tweet):
    tokens = nlp(tweet)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    
    return tokens

# vectorizer = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
vectorizer = CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))

##Doc2Vec

In [25]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
data = df_test

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [75]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [76]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("Hurrican on Atlanta Downtown".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-0.01947895 -0.02376492  0.0030167   0.01074679 -0.02430422  0.01346538
 -0.02337107  0.01762431  0.01169459  0.02275454 -0.01689866  0.01578643
  0.01763516 -0.0246702   0.00828856  0.02426583  0.01428954  0.01518091
 -0.00655923  0.00577711]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [77]:
# find most similar doc from the test data
mSimilar = model.docvecs.most_similar(positive=[model.infer_vector(test_data)],topn=5)
print("top 5 similar", mSimilar)

top 5 similar [('1', 0.4186919331550598), ('3', 0.31006813049316406), ('2', 0.09603632986545563), ('0', 0.014837533235549927)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [78]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('3', 0.2288547158241272), ('2', 0.06427853554487228), ('0', -0.3581893742084503)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [79]:
# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

[-0.02110824 -0.02011216 -0.00740983  0.02324364 -0.02904659 -0.02733524
  0.02049188 -0.01868427  0.00615    -0.00229033 -0.00763815  0.02084077
  0.00971974 -0.00894801  0.01055051  0.02820239 -0.00146638  0.00967818
  0.00186863  0.02681717]


##Training models

For our first attempts at building a model, we went back to models previously seen in the Data Mining & Machine Learning and Big-Scale Analytics courses, without tweaking the hyperparamaters too much, in order to compare how they each perform with this particular dataset.

Preparing the data: needs to be in the form of the list

In [0]:
X = vectorizer.fit_transform(df["text"].values.tolist())
y = df["target"].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

X_train

<6090x17410 sparse matrix of type '<class 'numpy.int64'>'
	with 51831 stored elements in Compressed Sparse Row format>

In [0]:
# vectorizer.get_feature_names()

### Random forest

In [0]:
RF = RandomForestClassifier(criterion="entropy", n_estimators=30, max_depth=200, random_state=RSEED)
RF.fit(X_train, y_train)
f1_score(y_test, RF.predict(X_test))

0.6904109589041096

### XGBoost

In [0]:
xgc = xgb.XGBClassifier(n_estimators=101, max_depth=71, base_score=0.5, objective='binary:logistic', random_state=RSEED)
xgc.fit(X_train, y_train)
f1_score(y_test, xgc.predict(X_test))

0.7341176470588235

### Neural network

### Logistic Regression

In [0]:
LR = LogisticRegressionCV(solver="lbfgs", cv=5, max_iter=1000, random_state=RSEED)
LR.fit(X_train, y_train)
f1_score(y_test, LR.predict(X_test))

0.7469084913437758

## Exporting predictions

In [0]:
To send our submissions for the Kaggle competition, we compute predictions for a set of tweets provided by the website and send them in a CSV file containing the tweet `id` and the predicted `target` 

In [0]:
X = vectorizer.transform(df_test["text"].values.tolist())
df_test["target"] = LR.predict(X)  # modify with appropriate model

df_test[["id", "target"]].to_csv("UNIL_SBB_FSP.csv", index=False)

## Unused code

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf',MultinomialNB()),])

text_clf.fit(Text_train, y_train)
print(text_clf.score(Text_test,y_test))

In [0]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import layers

input_dim = bow_train.shape[1] 
model = Sequential()
model.add(layers.Dense(600, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(200))
model.add(layers.Dense(150))
model.add(layers.Dense(10))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 512)               9479680   
_________________________________________________________________
dense_49 (Dense)             (None, 50)                25650     
_________________________________________________________________
dense_50 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_51 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 21        
Total params: 9,507,501
Trainable params: 9,507,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(bow_train, y_train, epochs=10, batch_size=50)
loss, accuracy = model.evaluate(bow_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0063 - accuracy: 0.9952
Epoch 2/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0061 - accuracy: 0.9947
Epoch 3/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9949
Epoch 4/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0062 - accuracy: 0.9956
Epoch 5/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9954
Epoch 6/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0063 - accuracy: 0.9952
Epoch 7/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9949
Epoch 8/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0065 - accuracy: 0.9954
Epoch 9/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9952
Epoch 10/10
6090/6090 [==============================] - 20s 3ms/step - l

In [0]:
loss, accuracy = model.evaluate(bow_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7663
